Install the required libraries

In [ ]:
# for cpu !pip3 install torch torchvision torchaudio
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 
!pip3 install transformers
!pip3 install lime
!pip3 install xformers
!pip3 install ipywidgets
!pip3 install scipy
!pip3 install pandas
!pip3 install numpy

Load the required libraries

In [ ]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import transformers
import pandas as pd
import torch
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

Code for Prediction and Explainability based on LIME

In [ ]:
# Check if CUDA is available; otherwise, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the XLNet model from the PT file
model_path = "model_checkpoint_epoch_BERT_Classifier_3.pt"  # Replace with the path to your PT file
model_state_dict = torch.load(model_path, map_location=device)
model = transformers.BertForSequenceClassification.from_pretrained("bert-base-cased", state_dict=model_state_dict)
model.to(device)

# Load the tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Load the dataset from CSV
csv_file = "IMDB_Reviews_Top_250_preprocessed_without_stopwords.csv"
df = pd.read_csv(csv_file)

In [ ]:
# Set the threshold for binary conversion
threshold = 7.0

# Convert the 'Rating' column to a binary variable
df['Binary Rating'] = df['Rating'].apply(lambda x: 1 if x >= threshold else 0)

In [ ]:
# Check which are the labels
class_names = ['Negative', 'Positive']

In [ ]:
def preprocess(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, max_length = 512, truncation=True)
    return inputs

In [ ]:
# Prediction function
def predict(text):
    inputs = preprocess(text)
    # Make predictions using the BERT model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
    return predictions.numpy()

In [ ]:
# Create a LIME explainer for text data
explainer = lime.lime_text.LimeTextExplainer(class_names=class_names)

def explain_prediction(text):
    # Make a prediction using the BERT model
    predicted_probabilities = predict(text)

    # Define the LIME explainer
    def classifier_fn(texts):
        return predict(texts)

    # Explain the prediction using LIME. If you want to show all the features, you have to add num_features=len(tokenizer.get_vocab())
    explanation = explainer.explain_instance(
        text, classifier_fn, num_features=10, num_samples=500
    )

    return explanation

Widget for adding a custom review

In [ ]:
# Create a loop for reviews that are in the dataset
for text in df['Review Text'][:2]:
    result = predict(text)
    explanation = explain_prediction(text)
    print("Predictions:", result)
    explanation.show_in_notebook(text=True)
    explanation.save_to_file('BERT_Classifier_XAI_LIME_Implementation.html')

In [ ]:
# Text input widget for the movie review
review_input = widgets.Text(
    value='This movie is great!',
    description='Movie Review:',
    layout=widgets.Layout(width='auto', flex='1 1 auto')
)

In [ ]:
# Button widget to trigger the function execution
button = widgets.Button(description='Lime Visualization', layout=widgets.Layout(width='auto', flex='0 0 auto'))

# Output widget to display the function output
output = widgets.Output()

In [ ]:
# Define the function to be executed on button click
def on_button_click(b):
    with output:
        clear_output()  # Clear the output cell before displaying new output
        your_movie_review = review_input.value
        result = predict(your_movie_review)
        explanation = explain_prediction(your_movie_review)
        print("Predictions:", result)
        explanation.show_in_notebook(text=True)
        explanation.save_to_file('BERT_Classifier_XAI_LIME_Implementation_Widget_Output.html')

button.on_click(on_button_click)

In [ ]:
# Create a label for the input field
label = widgets.Label(value='Enter your movie review:')

In [ ]:
# Arrange widgets in a responsive layout using HBox and VBox
input_box = widgets.VBox([label, review_input, button], layout=widgets.Layout(width='100%'))
display(widgets.VBox([input_box, output]))